<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import (TensorBoard, EarlyStopping, LearningRateScheduler, 
                                        ModelCheckpoint, CSVLogger, ReduceLROnPlateau)
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Version:  2.4.1


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [5]:
# Download and prepare the horses or humans dataset

splits, info = tfds.load('horses_or_humans', 
                         as_supervised=True, 
                         with_info=True, 
                         split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [6]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [7]:
def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return  image, label

In [8]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [9]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [10]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [11]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(dense_units, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')])
    return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [10]:
!rm -rf logs

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 7s 270ms/step - loss: 0.6755 - accuracy: 0.5720 - val_loss: 0.6594 - val_accuracy: 0.5122
Epoch 2/10
26/26 [==============================] - 5s 203ms/step - loss: 0.6253 - accuracy: 0.6554 - val_loss: 0.5770 - val_accuracy: 0.8537
Epoch 3/10
26/26 [==============================] - 6s 219ms/step - loss: 0.5614 - accuracy: 0.8024 - val_loss: 0.4889 - val_accuracy: 0.9512
Epoch 4/10
26/26 [==============================] - 7s 286ms/step - loss: 0.5216 - accuracy: 0.7521 - val_loss: 0.4158 - val_accuracy: 0.8976
Epoch 5/10
26/26 [==============================] - 8s 320ms/step - loss: 0.3991 - accuracy: 0.8921 - val_loss: 0.4096 - val_accuracy: 0.8488
Epoch 6/10
26/26 [==============================] - 8s 326ms/step - loss: 0.3637 - accuracy: 0.8787 - val_loss: 0.2473 - val_accuracy: 0.9707
Epoch 7/10
26/26 [==============================] - 8s 321ms/step - loss: 0.3098 - accuracy: 0.8904 - val_loss: 0.2007 - val_accuracy: 0.9707
Epoch 

In [12]:
%tensorboard --logdir logs

# You may need to access to http://localhost:6006/#scalars

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 4440.

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5
26/26 - 6s - loss: 0.6426 - accuracy: 0.6034 - val_loss: 0.5831 - val_accuracy: 0.7902

Epoch 00001: saving model to weights.01-0.58.h5
Epoch 2/5
26/26 - 5s - loss: 0.5516 - accuracy: 0.7311 - val_loss: 0.4940 - val_accuracy: 0.8146

Epoch 00002: saving model to weights.02-0.49.h5
Epoch 3/5
26/26 - 5s - loss: 0.5318 - accuracy: 0.7494 - val_loss: 0.5244 - val_accuracy: 0.7171

Epoch 00003: saving model to weights.03-0.52.h5
Epoch 4/5
26/26 - 6s - loss: 0.4208 - accuracy: 0.8345 - val_loss: 0.4250 - val_accuracy: 0.8146

Epoch 00004: saving model to weights.04-0.43.h5
Epoch 5/5
26/26 - 8s - loss: 0.3750 - accuracy: 0.8504 - val_loss: 0.3379 - val_accuracy: 0.8732

Epoch 00005: saving model to weights.05-0.34.h5


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])

26/26 - 5s - loss: 0.6751 - accuracy: 0.5852 - val_loss: 0.6677 - val_accuracy: 0.5024

Epoch 00001: saving model to saved_model
INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2
26/26 - 6s - loss: 0.6831 - accuracy: 0.5401 - val_loss: 0.7400 - val_accuracy: 0.4341

Epoch 00001: saving model to model.h5
Epoch 2/2
26/26 - 5s - loss: 0.6192 - accuracy: 0.6740 - val_loss: 0.5919 - val_accuracy: 0.8049

Epoch 00002: saving model to model.h5


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 6s - loss: 0.6713 - accuracy: 0.5815 - val_loss: 0.6586 - val_accuracy: 0.6390
Epoch 2/50
26/26 - 5s - loss: 0.6369 - accuracy: 0.6959 - val_loss: 0.6641 - val_accuracy: 0.5073
Epoch 3/50
26/26 - 6s - loss: 0.5898 - accuracy: 0.7360 - val_loss: 0.5720 - val_accuracy: 0.7902
Epoch 4/50
26/26 - 7s - loss: 0.5304 - accuracy: 0.7518 - val_loss: 0.5112 - val_accuracy: 0.7805
Epoch 5/50
26/26 - 8s - loss: 0.5011 - accuracy: 0.7810 - val_loss: 0.5859 - val_accuracy: 0.6732
Epoch 6/50
26/26 - 9s - loss: 0.4627 - accuracy: 0.7847 - val_loss: 0.5138 - val_accuracy: 0.7512
Epoch 7/50
26/26 - 10s - loss: 0.3947 - accuracy: 0.8370 - val_loss: 0.3817 - val_accuracy: 0.8829
Epoch 8/50
26/26 - 9s - loss: 0.3555 - accuracy: 0.8662 - val_loss: 0.3747 - val_accuracy: 0.8439
Epoch 9/50
26/26 - 9s - loss: 0.3426 - accuracy: 0.8662 - val_loss: 0.3249 - val_accuracy: 0.8878
Epoch 10/50
26/26 - 9s - loss: 0.2530 - accuracy: 0.9185 - val_loss: 0.1996 - val_accuracy: 0.9415
Epoch 11/50
26/26 

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 5s 201ms/step - loss: 0.7054 - accuracy: 0.5446 - val_loss: 0.6647 - val_accuracy: 0.5073
Epoch 2/5
26/26 [==============================] - 6s 215ms/step - loss: 0.6501 - accuracy: 0.5940 - val_loss: 0.6172 - val_accuracy: 0.6244
Epoch 3/5
26/26 [==============================] - 5s 210ms/step - loss: 0.5959 - accuracy: 0.6857 - val_loss: 0.6208 - val_accuracy: 0.6146
Epoch 4/5
26/26 [==============================] - 6s 233ms/step - loss: 0.6063 - accuracy: 0.6529 - val_loss: 0.6058 - val_accuracy: 0.6293
Epoch 5/5
26/26 [==============================] - 7s 256ms/step - loss: 0.5106 - accuracy: 0.7709 - val_loss: 0.4490 - val_accuracy: 0.8390


In [ ]:
pd.read_csv(csv_file).head()

## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])

In [ ]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

In [ ]:
%tensorboard --logdir log_dir